In [1]:
from itertools import islice

from matplotlib import pyplot as plt
import matplotlib.dates as mdates
from tqdm.autonotebook import tqdm

import torch
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.repository.datasets import get_dataset

from gluonts.dataset.pandas import PandasDataset
import pandas as pd

from lag_llama.gluon.estimator import LagLlamaEstimator

/tmp/ipykernel_193849/45153676.py:5: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
/home/abizard/miniconda3/envs/finetuning/lib/python3.12/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [3]:
ckpt = torch.load("./lag-llama-model/lag-llama.ckpt", map_location="cuda")
estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

estimator = LagLlamaEstimator(
    ckpt_path="./lag-llama-model/lag-llama.ckpt",
    prediction_length=12,
    context_length=32,

    # estimator args
    input_size=estimator_args["input_size"],
    n_layer=estimator_args["n_layer"],
    n_embd_per_head=estimator_args["n_embd_per_head"],
    n_head=estimator_args["n_head"],
    scaling=estimator_args["scaling"],
    time_feat=estimator_args["time_feat"],

    nonnegative_pred_samples=True,

    # linear positional encoding scaling
    rope_scaling={
        "type": "linear",
        "factor": max(1.0, (12 + 32) / estimator_args["context_length"]),
    },

    batch_size=64,
    num_parallel_samples=20,
)

lightning_module = estimator.create_lightning_module()
transformation = estimator.create_transformation()
predictor = estimator.create_predictor(transformation, lightning_module)

for name, p in lightning_module.named_parameters():
    print(f"Layer Name: {name} - Parameter Shape: {p.shape}")

/tmp/ipykernel_193849/4026714438.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("./lag-llama-model/lag-llama.ckpt", map_location="cuda")
/home/abizard

Layer Name: model.param_proj.proj.0.weight - Parameter Shape: torch.Size([1, 144])
Layer Name: model.param_proj.proj.0.bias - Parameter Shape: torch.Size([1])
Layer Name: model.param_proj.proj.1.weight - Parameter Shape: torch.Size([1, 144])
Layer Name: model.param_proj.proj.1.bias - Parameter Shape: torch.Size([1])
Layer Name: model.param_proj.proj.2.weight - Parameter Shape: torch.Size([1, 144])
Layer Name: model.param_proj.proj.2.bias - Parameter Shape: torch.Size([1])
Layer Name: model.transformer.wte.weight - Parameter Shape: torch.Size([144, 92])
Layer Name: model.transformer.wte.bias - Parameter Shape: torch.Size([144])
Layer Name: model.transformer.h.0.rms_1.scale - Parameter Shape: torch.Size([144])
Layer Name: model.transformer.h.0.attn.q_proj.weight - Parameter Shape: torch.Size([144, 144])
Layer Name: model.transformer.h.0.attn.kv_proj.weight - Parameter Shape: torch.Size([288, 144])
Layer Name: model.transformer.h.0.attn.c_proj.weight - Parameter Shape: torch.Size([144, 14